In [1]:
#Librerias
import pathlib
import ee
import geemap
import leafmap
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import cascaded_union
from pyproj import Proj, transform
import math
import time

In [2]:
#Crear directorios de entrada y salida de datos
project_name = 'PY_test'
plots_file_name = '41552-plot.shp'
directorio_actual = pathlib.Path.cwd()
directorio_inputs = directorio_actual / 'inputs'
directorio_salida = directorio_actual / 'salida' / project_name
directorio_salida_ceo = directorio_salida / 'ceo'
directorio_salida_planet = directorio_salida / 'planet_plots_serial'


directorio_inputs.mkdir(parents=True, exist_ok=True)
directorio_salida.mkdir(parents=True, exist_ok=True)
directorio_salida_ceo.mkdir(parents=True, exist_ok=True)
directorio_salida_planet.mkdir(parents=True, exist_ok=True)


In [3]:
# Cargar el archivo shapefile de puntos
grilla_plots = str(directorio_inputs) + '/' + plots_file_name

In [4]:
#Definir fechas para los mosaicos planet y dimension de las parcelas en pixeles (128,256,512)
fecha_mosaico = '2024-02-01'
dimensiones = 256

########################################################
tamanho_parcela = math.floor(dimensiones*4.77) #dato en metros


In [5]:
# Cargar el archivo shapefile de puntos
puntos = gpd.read_file(grilla_plots).set_crs(epsg=4326, inplace=True)

# Proyectar los puntos a metros
puntos_metros = puntos.to_crs(3857)

# Definir la función para crear un cuadrado de x metros alrededor de cada punto
def cuadrado_alrededor_punto(punto):
    x, y = punto.x, punto.y
    # Definir las coordenadas de los vértices del cuadrado
    coords = [(x - tamanho_parcela/2, y - tamanho_parcela/2), (x + tamanho_parcela/2, y - tamanho_parcela/2), (x + tamanho_parcela/2, y + tamanho_parcela/2), (x - tamanho_parcela/2, y + tamanho_parcela/2)]
    return Polygon(coords)

# Crear los cuadrados alrededor de cada punto
cuadrados = [cuadrado_alrededor_punto(Point(x, y)) for x, y in zip(puntos_metros.geometry.x, puntos_metros.geometry.y)]

# Crear un GeoDataFrame con los cuadrados
gdf_cuadrados = gpd.GeoDataFrame(geometry=cuadrados, crs="EPSG:3857")

# Copiar el atributo PLOTID al GeoDataFrame de los cuadrados
gdf_cuadrados['PLOTID'] = puntos['PLOT_ID']

gdf_cuadrados['PLOTID_str'] = gdf_cuadrados['PLOTID'].astype(str)

gdf_cuadrados_rp = gdf_cuadrados.to_crs(4326)

# Guardar el GeoDataFrame como un archivo shapefile
grilla_plots_cuadrados = str(directorio_salida_ceo) + '/' + 'plots_ceo_'+ project_name + '.shp.zip'
gdf_cuadrados_rp.to_file(grilla_plots_cuadrados, encoding='UTF-8')

#gdf_cuadrados.head()

In [6]:
#Cargar capas a GEE
geemap.ee_initialize()
poly_fc= geemap.geopandas_to_ee(gdf_cuadrados)

In [7]:
#Generar mosaicos de imagenes Planet
planet = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/americas");
planetList = ee.List([])

date2 = ee.Date(fecha_mosaico).advance(1,'day')
date1 = date2.advance(-3,'month')

def mosaicoPlanetLast (feature):
    planet_clip = planet.filterDate(date1, date2).sort('system:time_start', False).first().clip(feature.geometry())
    planetList.add(planet_clip)
    return planet_clip

def mosaicoPlanetLastDescarga (feature):
    planet_clip = planet.filterDate(date1, date2).sort('system:time_start', False).first().clip(feature.geometry())
    return planet_clip
   
mosaicoPlanet_clusters = ee.ImageCollection(poly_fc.map(mosaicoPlanetLast)).mosaic()

first_planet =planet.filterDate(date1, date2).sort('system:time_start', False).first()
planet_fecha = ee.String(first_planet.get('system:index')).slice(34, 41).replace('-','_').getInfo()
planet_layer_name = 'Imagen Planet ' + planet_fecha
print(planet_fecha)

2024_02


In [8]:
#Visualizacion de capas
Map = geemap.Map()
Map.centerObject(poly_fc, 6)
Map.addLayer(mosaicoPlanet_clusters, {'bands': ['R','G','B'],'min': 0,'max': 2000}, planet_layer_name)
Map

Map(center=[-25.82940937046416, -55.46298192863716], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
#Reescalar imagenes planet a 8bits
def reescalarPlanet(image):
    x = image.select(['R','G','B']).divide(5).toUint8()
    return x

In [ ]:
#Descarga de mosaicos planet en paralelo
geemap.download_ee_image_tiles_parallel(
    image= reescalarPlanet(mosaicoPlanet_clusters), 
    features=poly_fc,
    out_dir=str(directorio_salida_planet),
    scale=4.77, 
    crs="EPSG:3857",
    max_tile_dim=dimensiones,
    column='PLOTID_str' 
)

In [ ]:
#Descarga de mosaicos planet en serie
geemap.download_ee_image_tiles(
    image= reescalarPlanet(mosaicoPlanet_clusters),
    features=poly_fc,
    out_dir=str(directorio_salida_planet),
    scale=4.77,
    crs="EPSG:3857",
    max_tile_dim=dimensiones,
    column='PLOTID_str',
    num_threads = 4
)

In [ ]:
#Agregar parametros a archivo de texto
with open('parametros.txt', 'w') as file:
    file.write('proyect_name: '+project_name+ '\n' +
               'plots_file: '+plots_file_name+ '\n' +
               'fecha_mosaicos_planet: '+ fecha_mosaico+ '\n' +
               'parcel_side_pixels: '+ str(dimensiones)+ '\n' +
               'parcel_side_meters: '+ str(tamanho_parcela)+ '\n')